### RAG Test Notebook

#### openai_api_key can also be directly 

In [ ]:
import openai
import os

openai_api_key = os.environ.get('OPENAI_API_KEY')

#### Same for pinecone API key. I used apple-index as an example

In [18]:
from dspy import dspy
from dspy.retrieve.pinecone_rm import PineconeRM
import openai


# Although we didn't use local embedding model, dspy still requires us to install torch. Very weird.
try:
    __import__('torch')
except ImportError:
    pip.main(['install', 'torch'])    

llm = dspy.OpenAI(model="gpt-3.5-turbo")

pinecone_index_name = "apple-index"
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
pinecone_env = "us-east-1-aws"

retriever_model = PineconeRM(pinecone_index_name, 
                             pinecone_api_key, 
                             pinecone_env,
                             None, 
                             'text-embedding-ada-002',
                             openai_api_key,
                             None, 
                             5)
dspy.settings.configure(lm=llm, rm=retriever_model)

#### Testing if dspy connects to LMs (gpt-3.5 and gpt-4 are being used here)

In [3]:
print(dspy.settings.lm("Write a 3 line poem about neural networks."))
context_example = dspy.OpenAI(model="gpt-4")

with dspy.context(llm=context_example):
    print(context_example("Write a 3 line poem about neural networks."))

['Neural networks, a web of connections so vast,\nLearning and adapting, evolving fast,\nMimicking the human brain, a future forecast.']
['In the realm where data streams flow,\nNeural networks learn, adapt and grow,\nA dance of code, a silent show.']


#### Create a metric on a different model. This metric program is a recipe from [Weviate Recipe Github](https://github.com/weaviate/recipes/tree/main/integrations/dspy)

In [4]:
metricLM = dspy.OpenAI(model='gpt-4', max_tokens=1000, model_type='chat')

# Signature for LLM assessments.

class Assess(dspy.Signature):
    """Assess the quality of an answer to a question."""

    context = dspy.InputField(desc="The context for answering the question.")
    assessed_question = dspy.InputField(desc="The evaluation criterion.")
    assessed_answer = dspy.InputField(desc="The answer to the question.")
    assessment_answer = dspy.OutputField(desc="A rating between 1 and 5. Only output the rating and nothing else.")

def llm_metric(gold, pred, trace=None):
    predicted_answer = pred.answer
    question = gold.question
    
    print(f"Test Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    
    detail = "Is the assessed answer detailed?"
    faithful = "Is the assessed text grounded in the context? Say no if it includes significant facts not in the context."
    overall = f"Please rate how well this answer answers the question, `{question}` based on the context.\n `{predicted_answer}`"
    
    with dspy.context(lm=metricLM):
        retrieve = dspy.Retrieve(k=3)
        context = retrieve(question).passages
        detail = dspy.ChainOfThought(Assess)(context="N/A", assessed_question=detail, assessed_answer=predicted_answer)
        faithful = dspy.ChainOfThought(Assess)(context=context, assessed_question=faithful, assessed_answer=predicted_answer)
        overall = dspy.ChainOfThought(Assess)(context=context, assessed_question=overall, assessed_answer=predicted_answer)
    
    print(f"Faithful: {faithful.assessment_answer}")
    print(f"Detail: {detail.assessment_answer}")
    print(f"Overall: {overall.assessment_answer}")
    
    
    total = float(detail.assessment_answer) + float(faithful.assessment_answer)*2 + float(overall.assessment_answer)
    
    return total / 5.0

#### Quick Test

In [5]:
test_example = dspy.Example(question="What do cross encoders do?")
test_pred = dspy.Example(answer="They re-rank documents.")

type(llm_metric(test_example, test_pred))

Test Question: What do cross encoders do?
Predicted Answer: They re-rank documents.
Faithful: 1
Detail: 1
Overall: 3


float

#### Define a Signature for our RAG. Signature: "question, context ---> answer"

In [6]:
class RAGSignature(dspy.Signature):
    """Answer questions based on context. Answer I don't know if the question cannot be answered given the context."""
    
    question = dspy.InputField()
    context = dspy.InputField(desc="may contain relevant facts")
    answer = dspy.OutputField()

#### Define RAG Program. retrieves passages from our pinecone "apple-index" index.

In [7]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(RAGSignature)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        print(context)
        prediction = self.generate_answer(question=question, context=context)
        return dspy.Prediction(answer=prediction.answer)

#### Actual RAG Test

In [8]:
generate_answer = RAG(num_passages=10)

question = "What new products did apple release in each of the quarter this year? List the result as bullet points."
#retrieve = dspy.Retrieve(k = 3)
#topK_passages = retrieve(question).passages

pred = generate_answer(question=question)

print(f"Question: {question}")
print(f"Predicted Answer: {pred.answer}")

['Third Quarter 2023:\n\n• MacBook Air 15”, Mac Studio and Mac Pro;\n• Apple Vision Pro™, the Company’s first spatial computer featuring its new visionOS™, expected to be available in early calendar year 2024; and\n• iOS 17, macOS Sonoma, iPadOS 17, tvOS 17 and watchOS 10, updates to the Company’s operating systems.\n\nFourth Quarter 2023:\n\n• iPhone 15, iPhone 15 Plus, iPhone 15 Pro and iPhone 15 Pro Max; and\n• Apple Watch Series 9 and Apple Watch Ultra 2.']
Question: What new products did apple release in each of the quarter this year? List the result as bullet points.
Predicted Answer: - Third Quarter 2023:
  • MacBook Air 15”, Mac Studio and Mac Pro
  • Apple Vision Pro™
  • iOS 17, macOS Sonoma, iPadOS 17, tvOS 17 and watchOS 10

- Fourth Quarter 2023:
  • iPhone 15, iPhone 15 Plus, iPhone 15 Pro and iPhone 15 Pro Max
  • Apple Watch Series 9 and Apple Watch Ultra 2


### Inspect the query sent to OpenAI

In [9]:
llm.inspect_history(1)





Answer questions based on context. Answer I don't know if the question cannot be answered given the context.

---

Follow the following format.

Question: ${question}

Context: may contain relevant facts

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Question: What new products did apple release in each of the quarter this year? List the result as bullet points.

Context:
«Third Quarter 2023:

• MacBook Air 15”, Mac Studio and Mac Pro;
• Apple Vision Pro™, the Company’s first spatial computer featuring its new visionOS™, expected to be available in early calendar year 2024; and
• iOS 17, macOS Sonoma, iPadOS 17, tvOS 17 and watchOS 10, updates to the Company’s operating systems.

Fourth Quarter 2023:

• iPhone 15, iPhone 15 Plus, iPhone 15 Pro and iPhone 15 Pro Max; and
• Apple Watch Series 9 and Apple Watch Ultra 2.»

Reasoning: Let's think step by step in order to produce the answer. We need to look at the products released

## More to be added...